In [1]:
import requests
import json
import csv
import re
from Coordin_transformlat import gcj02towgs84

def Get_poi_polygon(key,polygon,keywords,page):
    '''
    这是一个能够从高德地图获取poi数据的函数
    key：为用户申请的高德密钥
    polygon：目标城市四个点的坐标
    keywords：POI数据的类型
    page：当前页数
    '''
    #设置header
    header = {'User-Agent': "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50"}

    #将输进来的矩形进行格式化
    Polygonstr = str(polygon[0]) + ',' + str(polygon[1]) + '|' + str(polygon[2]) + ',' + str(polygon[3])

    #构建url
    url = 'https://restapi.amap.com/v3/place/polygon?polygon={}&key={}&keywords={}&page={}'.format(Polygonstr, key, keywords, page)
    print(url)
    #用get函数请求数据
    r = requests.get(url, headers=header)

    #设置数据的编码为'utf-8'
    r.encoding = 'utf-8'

    # 将请求得到的数据按照'utf-8'编码成字符串
    data = r.text
    return data

def Get_times_polygon(key,polygon,keywords):
    '''
    这是一个控制Get_poi_polygon申请次数的函数
    '''
    page = 1
    # 执行以下代码，直到count为0的时候跳出循环
    while True:
        # 调用第一个函数来获取数据
        result = Get_poi_polygon(key,polygon, keywords, page)
        # json.loads可以对获取回来JSON格式的数据进行解码
        content = json.loads(result)
        pois = content['pois']
        count = content['count']
        print(count)
        #如果区域内poi的数量大于800，则认为超过上限，返回False请求对区域进行切割
        if int(count) > 800:
            return False
        else:
            for i in range(len(pois)):
                name = pois[i]['name']
                location = pois[i]['location']
                if 'address' not in pois[i].keys():
                    address = str(-1)
                else:
                    address = pois[i]['address']
                adname = pois[i]['adname']
                result = gcj02towgs84(location)
                lng = result[0]
                lat = result[1]
                row = [name, address, adname, lng, lat]
                #调用写入函数来保存数据
                writecsv(row, keywords)
            if count == '0':
                break
                # 递增page
            page = page + 1

def writecsv(poilist,keywords):
    """
    这是写入成csv文件的函数
    :param poilist:
    :param keywords:
    :return: 输出的结果存放在代码文件夹下
    """
    with open('{}.csv'.format(keywords),'a',newline='',encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(poilist)

def get_city_scope(key, cityname):
    parameters = 'key={}&keywords={}&subdistrict={}&output=JSON&extensions=all'.format(key, cityname, 0)
    url = 'https://restapi.amap.com/v3/config/district?'
    # 设置header
    header = {'User-Agent': "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50"}
    res = requests.get(url,params=parameters)
    jsonData = res.json()
    if jsonData['status'] == '1':
        district = jsonData['districts'][0]['polyline']
        district_list = re.split(';|\|',district)
        xlist, ylist = [], []
        for d in district_list:
            xlist.append(float(d.split(',')[0]))
            ylist.append(float(d.split(',')[1]))
        xmax = max(xlist)
        xmin = min(xlist)
        ymax = max(ylist)
        ymin = min(ylist)
        return [xmin, ymax, xmax, ymin]
    else:
        print ('fail to acquire: {}'.format(jsonData['info']))



In [2]:
# * 百度坐标系 (BD-09) 与 火星坐标系 (GCJ-02)的转换
# * 即 百度 转 谷歌、高德
# * @param bd_lon
# * @param bd_lat
# * @returns {*[]}
# */
import math
def bd09togcj02(bd_lon, bd_lat):
    x_pi = 3.14159265358979324 * 3000.0 / 180.0
    x = bd_lon - 0.0065
    y = bd_lat - 0.006
    z = math.sqrt(x * x + y * y) - 0.00002 * math.sin(y * x_pi)
    theta = math.atan2(y, x) - 0.000003 * math.cos(x * x_pi)
    gg_lng = z * math.cos(theta)
    gg_lat = z * math.sin(theta)
    return [gg_lng, gg_lat]


# * 火星坐标系 (GCJ-02) 与百度坐标系 (BD-09) 的转换
# * 即谷歌、高德 转 百度
# */
def gcj02tobd09(lng, lat):
    x_PI = 3.14159265358979324 * 3000.0 / 180.0
    z = math.sqrt(lng * lng + lat * lat) + 0.00002 * math.sin(lat * x_PI)
    theta = math.atan2(lat, lng) + 0.000003 * math.cos(lng * x_PI)
    bd_lng = z * math.cos(theta) + 0.0065
    bd_lat = z * math.sin(theta) + 0.006
    return [bd_lng, bd_lat]


# wgs84转高德
def wgs84togcj02(lng, lat):
    PI = 3.1415926535897932384626
    ee = 0.00669342162296594323
    a = 6378245.0
    dlat = transformlat(lng - 105.0, lat - 35.0)
    dlng = transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * PI
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * PI)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * PI)
    mglat = lat + dlat
    mglng = lng + dlng
    return [mglng, mglat]


# GCJ02/谷歌、高德 转换为 WGS84 gcj02towgs84
def gcj02towgs84(localStr):
    lng = float(localStr.split(',')[0])
    lat = float(localStr.split(',')[1])
    PI = 3.1415926535897932384626
    ee = 0.00669342162296594323
    a = 6378245.0
    dlat = transformlat(lng - 105.0, lat - 35.0)
    dlng = transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * PI
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * PI)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * PI)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng,lat * 2 - mglat]


def transformlat(lng, lat):
    PI = 3.1415926535897932384626
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * \
          lat + 0.1 * lng * lat + 0.2 * math.sqrt(abs(lng))
    ret += (20.0 * math.sin(6.0 * lng * PI) + 20.0 *
            math.sin(2.0 * lng * PI)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * PI) + 40.0 *
            math.sin(lat / 3.0 * PI)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * PI) + 320 *
            math.sin(lat * PI / 30.0)) * 2.0 / 3.0
    return ret


def transformlng(lng, lat):
    PI = 3.1415926535897932384626
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * math.sqrt(abs(lng))
    ret += (20.0 * math.sin(6.0 * lng * PI) + 20.0 *
            math.sin(2.0 * lng * PI)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * PI) + 40.0 *
            math.sin(lng / 3.0 * PI)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * PI) + 300.0 *
            math.sin(lng / 30.0 * PI)) * 2.0 / 3.0
    return ret

In [19]:
import GetPoi_keywords as gp

def Quadrangle(key,polygon,keywords):
    """
    :param key:高德地图密钥
    :param polygon: 矩形左上跟右下坐标的列表
    :param keywords: poi关键词
    :return:
    """
    #准备一个空列表，存放切割后的子区域
    PolygonList = []
    for i in range(len(polygon)):
        currentMinlat = round(polygon[i][0],6)#当前区域的最小经度
        currentMaxlat = round(polygon[i][2],6)#当前区域的最大经度
        currentMaxlon = round(polygon[i][1],6)#当前区域的最大纬度
        currentMinlon = round(polygon[i][3],6)#当前区域的最小纬度

        cerrnt_list = [currentMinlat, currentMaxlon, currentMaxlat, currentMinlon]
        #将多边形输入获取函数中，判断区域内poi的数量
        status = gp.Get_times_polygon(key,cerrnt_list,keywords)
        #如果数量大于800，那么返回False，对区域进行切分,否则返回区域的坐标对
        if status != False:
            print('该区域poi数量小于800，正在写入数据')
        else:
            #左上矩形
            PolygonList.append([
                currentMinlat, #左经
                currentMaxlon, #上纬
                (currentMaxlat+currentMinlat)/2, #右经
                (currentMaxlon+currentMinlon)/2]) #下纬
            #右上矩形
            PolygonList.append([
                (currentMaxlat+currentMinlat)/2,#左经
                currentMaxlon, #上纬
                currentMaxlat, #右经
                (currentMaxlon+currentMinlon)/2#下纬
            ])
            #左下矩形
            PolygonList.append([
                currentMinlat,#左经
                (currentMaxlon+currentMinlon)/2,#上纬
                (currentMaxlat+currentMinlat)/2,#右经
                currentMinlon#下纬
            ])
            #右下矩形
            PolygonList.append([
                (currentMaxlat+currentMinlat)/2,#左经
                (currentMaxlon+currentMinlon)/2,#上纬
                currentMaxlat,#右经
                currentMinlon#下纬
            ])
            print(len(PolygonList))
            if len(PolygonList) == 0:
                break
            else:
                Quadrangle(key,PolygonList,keywords)


#这里修改为自己的高德密钥
key ='61c256acd1b853e34a6b9a31033c399d'

#这里修改自己的poi类型
keywords = '公园广场'

#这里输入想要查询的城市
city = '上海市'

#调用高德查询行政区的API接口来返回矩形坐标对
Retance = gp.get_city_scope(key,city)

#存储区域矩形的列表
input_polygon = []
input_polygon.append(Retance)

Quadrangle(key,input_polygon,keywords)

https://restapi.amap.com/v3/place/polygon?polygon=120.856804,31.872716|122.247149,30.675593&key=61c256acd1b853e34a6b9a31033c399d&keywords=公园广场&page=1
515
https://restapi.amap.com/v3/place/polygon?polygon=120.856804,31.872716|122.247149,30.675593&key=61c256acd1b853e34a6b9a31033c399d&keywords=公园广场&page=2
515
https://restapi.amap.com/v3/place/polygon?polygon=120.856804,31.872716|122.247149,30.675593&key=61c256acd1b853e34a6b9a31033c399d&keywords=公园广场&page=3
515
https://restapi.amap.com/v3/place/polygon?polygon=120.856804,31.872716|122.247149,30.675593&key=61c256acd1b853e34a6b9a31033c399d&keywords=公园广场&page=4
515
https://restapi.amap.com/v3/place/polygon?polygon=120.856804,31.872716|122.247149,30.675593&key=61c256acd1b853e34a6b9a31033c399d&keywords=公园广场&page=5
515
https://restapi.amap.com/v3/place/polygon?polygon=120.856804,31.872716|122.247149,30.675593&key=61c256acd1b853e34a6b9a31033c399d&keywords=公园广场&page=6
515
https://restapi.amap.com/v3/place/polygon?polygon=120.856804,31.872716|122.2

In [17]:
import requests

url='https://www.sogou.com/'

response=requests.get(url=url)

page_text=response.text

with open('./sogou.html','w',encoding='utf-8') as fp:
    fp.write(page_text)

In [6]:

# -*- codeing = utf-8 -*-
# @Time : 2022/4/21 9:51
# @Author : zhang
# @File : WeiboSpider.py
# @Software : PyCharm
#引入包zzz
import urllib.parse
from urllib.parse import urlencode
import requests
import json
import time
from datetime import datetime
from pyquery import PyQuery as pq
import xlwt   #进行excel操作
​
​
​
#自定义区域
readjsonFile=json.load(open('config.json', 'r', encoding="utf-8"))
savepath = readjsonFile.get('savepath')     #可以自定义设计最终excel的路径
search =readjsonFile.get('search')                 #可以自定义设计查询什么内容
#设置起始时间
dateStart=readjsonFile.get('dateStart')    #日期格式必须是：形如：2022-4-21
dateEnd=readjsonFile.get('dateEnd')
​
​
​
urlsearch=urllib.parse.quote('=1&q='+search)     #进行url编码
m_referer='https://m.weibo.cn/search?containerid=100103type'   #微博搜索来源界面
base_url = 'https://m.weibo.cn/api/container/getIndex?'     #微博接口
#host用于指定internet主机和端口号，http1.1必须包含，不然系统返回400，
headers = {                 #封装请求头  让网站识别自己是浏览器
    'Referer': m_referer+urlsearch,             #告诉服务器自己是哪里来的  从那个页面来的   来路
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36 Edg/80.0.361.111',    #包含操作系统和浏览器信息
    'Host':'m.weibo.cn',                #请求服务器的域名和端口号
    'X-Requested-With':'XMLHttpRequest'             #代表是ajax请求
}
sign=0      #标志是否找到了实时微博以开始
Pqove=0     #标志位  表示是否读取文件结束
​
#获取网页的json（这个是获取搜索之后网页的json数据的函数）
def get_page(page):
    para={
        'containerid':m_referer[m_referer.find('100103'):]+urlsearch,
        'page':page
    }
    url = base_url+urlencode(para)                  #进行url编码添加到地址结尾  连带页数
    try:
        response = requests.get(url, headers=headers)       #request请求  地址和携带请求头
        if response.status_code == 200:
            return response.json()                              #以json格式返回数据
    except requests.ConnectionError as e:
        print("Error:",e.args)
​
#获取全文网页的json（这个是获取需要对全文获取的网页的json数据的函数）
def get_txt_page(containerid,referer):
    base_urlx = 'https://m.weibo.cn/statuses/show?'     #相较于获取搜索结果的url不通
    txtheaders = {
        'Referer': referer,
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36 Edg/80.0.361.111',
        'Host': 'm.weibo.cn',
        'X-Requested-With': 'XMLHttpRequest'
    }
    para = {
        'id': containerid,    #只需要一个参数  且参数名为id
    }
    url = base_urlx + urlencode(para)  # 进行url编码添加到地址结尾  连带页数
    try:
        response = requests.get(url, headers=headers)  # request请求  地址和携带请求头
        if response.status_code == 200:
            return response.json()  # 以json格式返回数据
    except requests.ConnectionError as e:
        print("Error:", e.args)
​
#分析JSON格式的数据，抓取目标信息
def parse_json(data):
    global sign                         #sign是用来标记是否有过title   在格式中从实时微博进行爬取  找到实时微博title之后开始爬取   之后不用进行判断title而用sign来判断
    if data:
        items = data.get('data').get('cards')
        for item1 in items:
            item=item1.get("title")
            if(item!=None or sign==1):       #分析数据发现只有实时微博开始时有一个title  所以可以进行判断是否有title并从title开始爬取
                item2=item1.get("card_group")
                # print(item2)
                # print(len(item2))
                for card in item2:
                    if card.get("mblog")!=None:             #如果没有mblog  那么就结束本次循环不尽兴数据爬取   如果有就进行爬取（因为会有一些数据中不包含mblog代表其并不是所需的数据）
                        sign=1
                        itemc=card.get("mblog")
                        weibo = {}
                        # 抓取信息
                        scheme=str(card.get("scheme"))              #获取具体博文链接（可以访问具体博文数据）
                        txtwb=pq(itemc.get('text')).text()          #获取微博博文
                        if  txtwb.find("全文")+2==len(txtwb):         #利用微博博文进行判断是否结尾有“全文二字”
​
                            datatxt = get_txt_page(scheme[scheme.find('mblogid=')+8:scheme.find('mblogid=')+17],scheme)  # 如果有全文那么需要进入其中再次进行爬取全文数据，首先切割链接获取mblogid，scheme对应的是Referer的链接信息
​
                            txtitem=pq(datatxt.get('data').get('text')).text()
​
                            weibo['text'] = pq(datatxt.get('data').get('text')).text()          #用pyquery去处理得到的数据
                        else:
                            weibo['text'] = pq(itemc.get('text')).text()        #不需要对全文做处理直接获取text
                        weibo['date'] = str(datetime.strptime(pq(itemc.get('created_at')).text(), '%a %b %d %H:%M:%S +0800 %Y'))#日期   #修改日期格式   默认微博日期格式是带时区的GMT的格式
                        weibo['attitudes'] = itemc.get('attitudes_count')  # 点赞次数
                        weibo['comments'] = itemc.get('comments_count')  # 评论次数
                        weibo['reposts'] = itemc.get('reposts_count')  # 转发次数
                        weibo['userid']=itemc.get('user').get('id')   #发布人id
                        weibo['username'] = itemc.get('user').get('screen_name')  # 发布人微博名
                        # 一个一个返回weibo
                        yield weibo
            else:
​
                continue
​
#存储到excel中
saveB=1             #标志位用来标记以及存储了多少条数据  还可以在存储时进行行数的标记
datelist=["userid", "username", "date", "attitudes", "comments", "reposts", "text"]
​
#全局定义excel中的相关参数   因为涉及多次的进行存入工作
book = xlwt.Workbook(encoding="utf-8", style_compression=0)             # 创建workbook对象
sheet = book.add_sheet('微博相关话题博文信息', cell_overwrite_ok=True)      # 创建工作表
col = ("发布人id", "发布人微博名", "发布时间", "点赞次数", "评论次数", "转发次数", "博文内容")
for i in range(0, 7):
    sheet.write(0, i, col[i])    #设置第一行显示内容
#存储到excel中
def save_data_toexcel(datas,savepath):
    global Pqove
    global saveB
    global sheet
    global book
​
    print("!!!!!!!!存入数据中")
    if  Pqove==1:               #对数据读取全部完成之后存入excel中
        print("存入数据完成！！！！！！！")
        book.save(savepath)
        return
​
    for j in range(0, 7):
        sheet.write(saveB, j, datas.get(datelist[j]))       #存储数据  saveB用来计数也是标记行数
    print("已存入%d条数据" % saveB)
    saveB+=1
​
​
def main():      #main函数
    global Pqove
    bio = 0                 #bio 是用来标记是否连续多个超出规定时间段的博文（连续超过3个就认为博文已经爬取完毕）
    page = 1                #页数  从第一页开始  也是需要传入的一个参数
    while True:
        data = get_page(page)                   #获取网页的json格式的数据
        results = parse_json(data)              #解析网页的json数据
        for result in results:                  #循环去便利数据
            # print(type(result['date']))
            d1 = datetime.strptime(result['date'], '%Y-%m-%d %H:%M:%S') #博文时间
            d2 = datetime.strptime(dateEnd+' 00:00:00', '%Y-%m-%d %H:%M:%S')           #结束时间
            d3 = datetime.strptime(dateStart+' 00:00:00', '%Y-%m-%d %H:%M:%S')          #开始时间
            # print(d1)
            # print(d1<d2)
            if d1 >= d3:
                                   #逻辑：  判断时间是否在开始和结束时间之间 如果在那么就存入数据   如果不在修改bio的数据   page++   如果bio达到3那么跳出while
                if d1 <= d2:
                    bio = 0
                    save_data_toexcel(result,savepath)
                    # save_date_tolist(result,savepath)
                    # save_data(result, page)  # 判断是否存入（要求必须是规定时间之内的微博）   判定逻辑：如果出现连续三个时间超过的规定范围那么便停止抓取
                else:
                    bio=0
            else:
                bio += 1
                if bio>=10:
                    Pqove=1
                    save_data_toexcel(result,savepath)
                    break
                    # save_date_tolist(result,savepath)
        print('第' + str(page) + '页抓取完成')
        # print(bio)
        if bio>=10:
            break
        page += 1
​
if __name__ =='__main__':
    main()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 101)